In [ ]:
%pip install -U insightface onnxruntime-gpu  numpy

In [ ]:
%pip install -U gdown moviepy ffmpeg insightface natsort

In [ ]:
!sudo apt install ffmpeg

In [ ]:
import gdown
import cv2
import os
from moviepy.editor import VideoFileClip,AudioFileClip
import glob
import os.path as osp
import shutil
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

In [ ]:

# Define variables
drive_file_id = '1wX74RzcHlAlYIHBJxH4lp0O-hXjCrUtR'  # replace 'FILE_ID' with your file's ID
video_output = 'my_video.mp4'
frame_folder = '/notebooks/frame_folder'
swapped_folder = '/notebooks/swapped'

In [ ]:
shutil.rmtree(frame_folder)
shutil.rmtree(swapped_folder)

In [ ]:
assert insightface.__version__ >= '0.7'


# Create the frame folder if it doesn't exist
if not os.path.exists(frame_folder):
    os.makedirs(frame_folder)
# Create the frame folder if it doesn't exist
if not os.path.exists(swapped_folder):
    os.makedirs(swapped_folder)
# Your face swapping script
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(640, 640))
swapper = insightface.model_zoo.get_model('/notebooks/inswapper_128.onnx')

In [ ]:


import numpy as np
def is_similar(face1, face2, threshold=0.5):
    # Calculate the distance between the embeddings of the two faces
    similarity = np.dot(face1.embedding, face2.embedding) / (np.linalg.norm(face1.embedding) * np.linalg.norm(face2.embedding))

    # If the distance is less than the threshold, the faces are considered similar
    return similarity > threshold
def get_first_face(image):
    # If the image is a string (presumably a file path), read the image
    if isinstance(image, str):
        image = cv2.imread(image)

    # Check if the image is a valid numpy array
    if isinstance(image, np.ndarray):
        source_faces = app.get(image)
        source_faces = sorted(source_faces, key=lambda x: x.bbox[0])
        if len(source_faces) == 0:
            return None
        return source_faces[0]
    return None

In [ ]:
import requests

url = "http://example.com/path_to_your_file"  # replace with your file's URL
response = requests.get(url)

with open(video_output, 'wb') as f:  # replace with the path where you want to save the file
    f.write(response.content)

In [ ]:
# Download video from Google Drive
url = f'https://drive.google.com/uc?id={drive_file_id}'
gdown.download(url, video_output, quiet=False)

In [ ]:

# Extract frames from video
vidcap = cv2.VideoCapture(video_output)
success, image = vidcap.read()
count = 0
target_imgs = []  # a list of file paths to images of the target faces
while success:
    path = os.path.join(frame_folder, f"{count}.jpg")
    target_imgs.append(path)
    cv2.imwrite(path, image)  # save frame as JPEG file
    success, image = vidcap.read()
    count += 1

In [ ]:
shutil.rmtree(swapped_folder)

In [ ]:
# Create the frame folder if it doesn't exist
if not os.path.exists(swapped_folder):
    os.makedirs(swapped_folder)

In [ ]:
import random
import natsort

swappings ={
   'm':get_first_face('/notebooks/1629085462605.jpeg'),
   'b':get_first_face('/notebooks/bcp.png'),
   'd':get_first_face('/notebooks/DSCAAZZEA.png'),
   'l':get_first_face('/notebooks/lcp.jpg'),
   'l2':get_first_face('/notebooks/2023-07-03 22.46.08.jpg'),
   'la':get_first_face('/notebooks/2023-07-03 22.47.10.jpg'),
   'jo':get_first_face('/notebooks/jocp.png'),
   'g':get_first_face('/notebooks/gacp.png'),
   'je':get_first_face('/notebooks/2023-07-03 22.47.24.jpg')
}
targets={

}

source_face = get_first_face('/notebooks/DSC06729.JPG')
image_files = glob.glob(os.path.join(frame_folder, '*.jpg'))  # get all jpg files in the frame folder
# Sort the file names using natural sorting
image_files = natsort.natsorted(image_files)
print(len(image_files))
# Initialize an empty dictionary to store the attribution of faces to people
attribution = {}
# Process each image and save the result
for img_file in image_files:
    img = cv2.imread(img_file)
    faces = app.get(img)
    faces = sorted(faces, key=lambda x: x.bbox[0])
    res = img.copy()

    for face in faces:
        found = False
        for key, value in targets.items():
                if(is_similar(face,value,0.6)):
                    found = True
                    res = swapper.get(res, face, swappings[key], paste_back=True)
                    break   
        if not found:
            # Check if the face is already attributed to a person
            for person, person_face in attribution.items():
                if is_similar(face, person_face,0.6):
                    attribution[person] = person_face
                    found = True
                    res = swapper.get(res, face, swappings[person], paste_back=True)
                    break
        if not found:
            keys = list(swappings.keys())
            for key in list(targets.keys()):
                keys.remove(key)
            random_key = random.choice(keys)
            attribution[random_key] = face
            random_value = swappings[random_key]
            res = swapper.get(res, face, random_value, paste_back=True)
    cv2.imwrite(osp.join(swapped_folder, osp.basename(img_file)), res)



In [ ]:
import subprocess

output_video = "/notebooks/output.mp4"
video = cv2.VideoCapture(video_output)
fps = video.get(cv2.CAP_PROP_FPS)
video.release()
print('frames per second =',fps)
# Get the processed images
processed_images = glob.glob(os.path.join(swapped_folder, '*.jpg'))

# Sort the processed images (this may be necessary depending on how your files are named)
processed_images.sort()
print(len(processed_images))
# Initialize the video writer
height, width, _ = cv2.imread(processed_images[0]).shape
print(height, width)

# Define the command
command = f'ffmpeg -y -r {fps} -s {width}x{height} -i {swapped_folder}/%01d.jpg -vcodec libx264 -crf 25 -pix_fmt yuv420p {output_video}'

# Execute the command
subprocess.call(command, shell=True)

In [ ]:
import subprocess


temp_audio_file = '/notebooks/temp_audio.aac'

# Remove temporary audio file if it exists
if os.path.exists(temp_audio_file):
    os.remove(temp_audio_file)

# Remove output video with audio file if it exists
output_with_audio_file = '/notebooks/output_with_audio.mp4'
if os.path.exists(output_with_audio_file):
    os.remove(output_with_audio_file)

# Extract audio from original video and save it as a temporary audio file
audio_extraction_command = f'ffmpeg -y -i {video_output} -vn -acodec aac -strict -2 {temp_audio_file}'
subprocess.run(audio_extraction_command, shell=True)

# Combine swapped video with original audio
video_combination_command = f'ffmpeg -y -i {output_video} -i {temp_audio_file} -c:v copy -c:a copy -map 0:v:0 -map 1:a:0 {output_with_audio_file}'
subprocess.run(video_combination_command, shell=True)